> ### __Otimização por Média-Variância__ (Teoria Moderna de Portfolios)
> - Desenvolvido por **Markowitz** em 1952 é a primeira tentativa de uma abordagem quantitativa para alocação de ativos. De forma simples, Markowitz traduz o velho ditado para o mundo das finanças:
> <center>
> <i>Não coloque todos os ovos em uma mesma cesta.</i>
> </center>
> 
> - Apesar de ser um modelo super teórico (e que não funciona bem na prática, como veremos), serve de base para modelos de alocação de ativos mais avançados (Black-Litterman, Hierarchical Risk Parity).  
> #### __Definições__:
> - A principal pergunta de Markowitz é a seguinte: 
><center> Como devemos alocar nossos ativos de tal forma que minimizamos o risco?</center>
>
> - O que, de fato, é risco? No contexto de Markowitz:
>
> <center> Risco = Variância na Distribuição de Retornos </center>
>
>- Ou seja, se o retorno do ativo i é uma variável aleatória $R_i$: 
> $$Risco = var(R_i)$$
> - Há outras formas de medir o risco de um certo ativo (ver: VaR, CVar, Negative Variance...)
>
> ##### __Caso 2 ativos:__
> - Suponha que tenhamos dois ativos (PETR4 e VALE3, por exemplo) e que tenhamos uma certa quantia pra investir.
> - Seja $\omega_1$ e $\omega_2$ a fração da quantia que você irá investir no primeiro e no segundo ativo, respectivamente, 
> - Como você quer investir tudo, $$\omega_1 + \omega_2 = 1$$
> - O retorno da sua carteira seria, então: $$R = \omega_1 R_1  +\omega_2 R_2  $$
> - Pra facilitar, vamos usar uma notação vetorial: 
>  $$R_{carteira} = \omega^T R =\begin{bmatrix} \omega_1 & \omega_2 \end{bmatrix} \begin{bmatrix} R_1 \\ R_2 \end{bmatrix}$$
> -  Para o Retorno Esperado, basta aplicarmos o operado $\mathbb{E}$
> - E a variância, depois de algumas contas:
> $$\sigma^2 = \omega^T \Sigma \omega =  \begin{bmatrix} \omega_1 & \omega_2 \end{bmatrix} \begin{bmatrix} \sigma^2_1 & \rho \sigma_1 \sigma_2 \\ \rho \sigma_1 \sigma_2 & \sigma^2_2   \end{bmatrix} \begin{bmatrix} \omega_1 \\ \omega_2 \end{bmatrix} $$
> - Onde o sigma é a matriz de variância-covariância.
> - Suponha então, que dadas essas equações, alguém lhe peça para encontrar a carteira com menor risco. Ou seja, dado que $\omega_1$ e $\omega_2$ somam 1, como encontrar uma combinação que minimize $\sigma^2$?
> - Solução __"Fácil"__: varrer todos os valores de $\omega_1$ e encontrar o mínimo. 
> - Solução __"Difícil"__: Cálculo? Otimização? Lagrange?   

In [1]:
### Funções que calculam o risco e o retorno de uma certa carteira 
import numpy as np
import matplotlib.pyplot as plt

def portfolio_return(omega:np.ndarray, expected_returns: np.ndarray)->float:
    return omega.T@expected_returns

def portfolio_variance(omega:np.ndarray, sigma: np.ndarray)->float:
    return omega.T@sigma@omega

def efficient_frontier_2assets(expected_returns:np.ndarray, sigma: np.ndarray, steps:int = 100)->(np.ndarray, np.ndarray):
    omega_1 = np.linspace(0, 1, steps)
    omega = np.vstack([omega_1, 1-omega_1])

    portfolios_returns = portfolio_return(omega, expected_returns).ravel()
    portfolios_std = np.sqrt(portfolio_variance(omega, sigma).diagonal()) 

    return portfolios_returns, portfolios_std

In [3]:
import ipywidgets as widgets
rho = widgets.FloatSlider(value = 0, min=-1, max = 1, step =0.01, description = 'Correlation: ', readout_format='.3f')
sigma_1 = widgets.FloatSlider(value = 0.01, min=0.001, max = 0.02, step =0.001, description = 'Std Asset 1: ', readout_format='.3f')
sigma_2 = widgets.FloatSlider(value = 0.02, min=0.001, max = 0.02, step =0.001, description = 'Std Asset 2: ', readout_format='.3f')

r_1 = widgets.FloatSlider(value = 0.01, min=0.001, max = 0.02, step =0.001, description = 'R Asset 1: ', readout_format='.3f')
r_2 = widgets.FloatSlider(value = 0.015, min=0.001, max = 0.02, step =0.001, description = 'R Asset 2: ', readout_format='.3f')

box_returns = widgets.HBox([r_1, r_2])
box_var = widgets.HBox([sigma_1, sigma_2])

box = widgets.VBox([box_returns, box_var, rho])

def plot_efficient_frontier(r_1, r_2, sigma_1, sigma_2, rho):
    expected_returns = np.array([[r_1], [r_2]])
    sigma = np.array([[sigma_1**2, rho*sigma_1*sigma_2], 
                      [rho*sigma_1*sigma_2, sigma_2**2]])

    portfolios_returns, portfolios_std = efficient_frontier_2assets(expected_returns, sigma, 100)
    # with plt.xkcd():
    fig, ax = plt.subplots(figsize=(16, 6))
    plt.plot(portfolios_std, portfolios_returns, linestyle = '--', color = 'firebrick')
    plt.title("Fronteira Eficiente - 2 Ativos")
    plt.xlim(0, 0.021)
    plt.ylim(0, 0.021)
    plt.xlabel('Risco')
    plt.ylabel('Retorno Esperado')
    plt.grid()
            
out = widgets.interactive_output(plot_efficient_frontier,  {'r_1': r_1,
                                                            'r_2': r_2, 
                                                            'sigma_1': sigma_1, 
                                                            'sigma_2': sigma_2, 
                                                            'rho': rho})

display(out, box)

Output()